# Getting Started with the NeMo Agent Toolkit

In this notebook, we walk through the basics of using the toolkit, from installation all the way to creating and running your very own custom workflow.

## Environment Setup

Ensure you meet the following prerequisites:
1. Git
2. [uv](https://docs.astral.sh/uv/getting-started/installation/)
3. NeMo-Agent-Toolkit installed from source following [these instructions](https://github.com/cdgamarose-nv/NeMo-Agent-Toolkit/tree/develop?tab=readme-ov-file#install-from-source)


### Set API keys

In [ ]:
import getpass
import os

if "NVIDIA_API_KEY" not in os.environ:
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key

if "TAVILY_API_KEY" not in os.environ:
    tavily_api_key = getpass.getpass("Enter your Tavily API key: ")
    os.environ["TAVILY_API_KEY"] = tavily_api_key

if "OPENAI_API_KEY" not in os.environ:
    openai_api_key = getpass.getpass("Enter your OpenAI API key: ")
    os.environ["OPENAI_API_KEY"] = openai_api_key

## Bringing an Agent into the NeMo-Agent-Toolkit

NeMo Agent toolkit works side-by-side and complements any existing agentic framework or memory tool you're using and isn't tied to any specific agentic framework, long-term memory, or data source. This allows you to use your current technology stack - such as LangChain, LlamaIndex, CrewAI, and Microsoft Semantic Kernel, as well as customer enterprise frameworks and simple Python agents - without replatforming.

We'll walk you through how to achieve this.

To demonstrate this, let's say that you have the following simple langchain agent that answers generic user queries about current events by performing a web search using Tavily. We will show you how to bring this agent into the NeMo-Agent-Toolkit and benefit from the configurability, resuability, and easy user experience.

Run the following two cells to create the langchain agent and run it with an example input.

In [ ]:
%load langchain_sample/langchain_agent.py

In [ ]:
!python langchain_sample/langchain_agent.py

#### Creating a new NeMo-Agent-Toolkit Workflow 

Bringing this agent into the toolkit requires creating a new workflow and configuring the tools, and so on. A workflow is a self-contained pipeline that orchestrates tools (e.g., custom arithmetic tools, web search, RAG) and one or more LLMs to process user inputs and generate outputs.

With our `nat workflow create` sub-command, you can scaffold and register new workflows within seconds. 

For example, to create an agent called `first_search_agent` in `.tmp/notebooks` you would run the following commands. 

> Note: The agent in this example has already been created in `examples/notebooks/first_search_agent` directory.

```bash
mkdir -p $PROJECT_ROOT/.tmp/notebooks
nat workflow create --workflow-dir $PROJECT_ROOT/.tmp/notebooks/first_search_agent
```

Expected Cell Output:
```bash
Installing workflow 'first_search_agent'...
Workflow 'first_search_agent' installed successfully.
Workflow 'first_search_agent' created successfully in '/NeMo-Agent-Toolkit/.tmp/notebooks/first_search_agent'.
```


The above command:
- Creates a new directory similar to `examples/notebooks/first_search_agent`.
- Sets up the necessary files and folders.
- Installs the new Python package for your workflow.

The registration process is built around two main components:
1. **A configuration class that inherits from `WorkflowBaseConfig`**
    
    Configuration classes that inherit from `TypedBaseModel` and `BaseModelRegistryTag` serve as Pydantic-based configuration objects that define both the plugin type identifier and runtime configuration settings for each NeMo Agent toolkit component. Each plugin type (functions, LLMs, embedders, retrievers, memory, front-ends, etc.) has its own base configuration class (e.g., `FunctionBaseConfig`, `LLMBaseConfig`, `EmbedderBaseConfig`) that establishes the plugin category, while concrete implementations specify a unique name parameter that automatically populates the type field for plugin identification. These configuration classes encapsulate runtime parameters as typed Pydantic fields with validation rules, default values, and documentation (e.g., `api_key`, `model_name`, `temperature` for LLM providers, or `uri`, `collection_name`, `top_k` for retrievers), enabling type-safe configuration management, automatic schema generation, and validation across the entire plugin ecosystem.

2. **A decorated async function (with `@register_workflow`) that yields a callable response function.**
     
     A `FunctionInfo` object is a structured representation yielded from functions decorated with `@register_function` that serves as a framework-agnostic wrapper for callable functions in the NeMo Agent Toolkit. This object encapsulates the function's main callable (e.g., `_response_fn`) that will be invoked at runtime, along with its input/output Pydantic schemas for validation, description for documentation, and optional type converters for automatic type transformation. FunctionInfo objects provide a consistent interface that can be dynamically translated into framework-specific representations (e.g., LangChain tools, LlamaIndex functions) at runtime or invoked directly as standard Python async coroutines, enabling seamless integration across different LLM frameworks while maintaining type safety and validation.


Once configured, you can run workflows via the command line (`nat run`) or launch them as services (`nat serve`) to handle requests in real time.

#### Customizing your Workflow

Now its time to define the same langchain agent inside your newly created workflow. This is as simple as making a few code additions to the `first_search_agent_function`.
- Add langchain framework wrappers (all this does is indicate which framework you are wrapping your code in which enables profiling the workflow later)
- Paste your agent initialization code inside the `first_search_agent_function`
- Paste your agent invocation code inside the `_response_fn` function

Your final `first_search_agent_function.py` should look like:

In [ ]:
%load first_search_agent/src/nat_first_search_agent/first_search_agent_function.py

Once you have your workflow registered, you can reference it by its `_type` in a YAML file. 

For example:

```yaml
workflow:
  _type: first_search_agent
```

#### Running your Workflow

The NeMo Agent toolkit provides several ways to run/host an workflow. These are called `front_end` plugins. Some examples are:

console: `nat run` (or long version nat start console …). This is useful when performing local testing and debugging. It allows you to pass inputs defined as arguments directly into the workflow. This is show already in the notebook.

Fastapi: `nat serve`(or long version nat start fastapi …). This is useful when hosting your workflow as a REST and websockets endpoint.

MCP: `nat mcp` (or long version nat start mcp …). This is useful when hosting the workflow and/or any function as an MCP server

While these are the built in front-end components, the system is extensible with new user defined front-end plugins.

For more info, here is a good resource for using the various plugins from the CLI: [cli.md](https://github.com/NVIDIA/NeMo-Agent-Toolkit/blob/develop/docs/source/reference/cli.md)

In order to test your new agent using the console, run:

In [ ]:
!nat run --config_file first_search_agent/configs/config.yml --input "Who is the current Pope?"

As shown above, this will return the same output as your previously created langchain agent.

#### Runtime Configurations

To benefit from the configurability of this toolkit, we can update the configuration object and config file along with the function to use the parameters at runtime.

This involves allowing the toolkit to sets up your tools, LLM, and any additional logic like maximum number of historical messages to provide to the agent, maximum number of iterations to run the agent, description of the agent and so on.

The toolkit will make use of the `Builder` class to utilize them at runtime.

Your final configuration object should look like this:
```python
class SecondSearchAgentFunctionConfig(FunctionBaseConfig, name="second_search_agent"):
    """
    NeMo Agent toolkit function template. Please update the description.
    """
    tool_names: list[FunctionRef] = Field(default=[], description="List of tool names to use")
    llm_name: LLMRef = Field(description="LLM name to use")
    max_history: int = Field(default=10, description="Maximum number of historical messages to provide to the agent")
    max_iterations: int = Field(default=15, description="Maximum number of iterations to run the agent")
    handle_parsing_errors: bool = Field(default=True, description="Whether to handle parsing errors")
    verbose: bool = Field(default=True, description="Whether to print verbose output")
    description: str = Field(default="", description="Description of the agent")
```

You can then replace:
```python
tool = [search]
```
with 
```python
tools = builder.get_tools(config.tool_names, wrapper_type=LLMFrameworkEnum.LANGCHAIN)
```
> **Note**: This allows you to bring in tools from other frameworks like llama index as well and wrap them with langchain since you are implementing your agent in langchain.

In a similar way, you can initialize your llm by utilizing the parameters from the configuration object in the following way:
```python
llm = await builder.get_llm(config.llm_name, wrapper_type=LLMFrameworkEnum.LANGCHAIN)
```

For each tool or reusable plugin, there are potentially multiple optional parameters with default values that can be overridden. The `nat info components` command can be used to list all available parameters. For example, to list all available parameters for the LLM nim type run:

```bash
nat info components -t llm_provider -q nim
```

#### Reusing the Inbuilt Tavily Search Function

We can also make use of some of many example functions that the toolkit provides for common use cases. In this agent example, rather than reimplementing the tavily search, we will use the inbuilt function for internet search which is built on top of langchain's tavily search API. You can list available functions using the following:

In [ ]:
!nat info components -t function -q tavily_internet_search

This function can be used any number of times in the configuration YAML by specifying the `_type` as `tavily_internet_search`

```yaml
functions:
  my_internet_search:
    _type: tavily_internet_search
    max_results: 2
    api_key: $TAVILY_API_KEY
```

#### Final Code and Configuration
The final code for your workflow can be found in [this example](examples/my_agent_workflow/src/nat_my_agent_workflow/my_agent_workflow_function.py)

In [ ]:
%load first_search_agent/src/nat_first_search_agent/second_search_agent_function.py

The final configuration file should resemble the following:

In [ ]:
%load first_search_agent/configs/config_modified.yml

In [ ]:
!nat run --config_file first_search_agent/configs/config_modified.yml --input "Who is the current Pope?"

#### NAT Serve

You can also use the `nat serve` sub-command to launch a server and make HTTP requests to the endpoints as shown below. Refer to [this documentation](https://docs.nvidia.com/nemo/agent-toolkit/latest/reference/api-server-endpoints.html) for more information on available endpoints.

In [ ]:
%%bash --bg
# This will start background nat service and might take a moment to be ready
nat serve --config_file first_search_agent/configs/config_modified.yml

In [ ]:
%%bash
# Issue a request to the background service
curl --request POST \
  --url http://localhost:8000/chat \
  --header 'Content-Type: application/json' \
  --data '{
    "messages": [
      {
        "role": "user",
        "content": "Who is the current Pope?"
      }
    ]
}'
# Terminate the process after completion
pkill -9 nat

#### Reusing the Inbuilt ReAct Agent

NeMo Agent Toolkit has a reusable react agent function. We can reuse that agent here to simplify the workflow even further.

In [ ]:
!nat info components -t function -q react_agent

In [ ]:
%load first_search_agent/configs/config_react_agent.yml

In [ ]:
!nat run --config_file first_search_agent/configs/config_react_agent.yml --input "Who is the current Pope?"